In [7]:
import pygame
import pygame.joystick
import math

# Initialize Pygame
pygame.init()
pygame.joystick.init()

# Set up the display
WIDTH = 1000
HEIGHT = 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("PS5 Controller Dot Movement")

# Colors
WHITE = (255, 255, 255)
GRAY = (128, 128, 128)
BLACK = (0, 0, 0)
BLUE = (0, 140, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

class DotEnvironment:
    def __init__(self):
        self.controller = None
        self.connect_controller()
        
        # Dot properties
        self.dot_pos = [WIDTH // 2, HEIGHT // 2]  # Start in center
        self.dot_radius = 10
        self.dot_speed = 1
        self.dot_color = RED
        
        # Movement properties
        self.velocity = [0, 0]
        self.acceleration = 0.5
        self.friction = 0.9
        
    def connect_controller(self):
        if pygame.joystick.get_count() > 0:
            self.controller = pygame.joystick.Joystick(0)
            self.controller.init()
            print(f"Connected to: {self.controller.get_name()}")
        else:
            print("No controller found!")
    
    def update_dot_position(self):
        if self.controller:
            # Get analog stick values
            left_x = self.controller.get_axis(0)
            left_y = self.controller.get_axis(1)
            right_x = self.controller.get_axis(2)
            right_y = self.controller.get_axis(3)
            
            # Apply deadzone to prevent drift
            deadzone = 0.1
            if abs(left_x) < deadzone: left_x = 0
            if abs(left_y) < deadzone: left_y = 0
            
            # Update velocity based on left stick
            self.velocity[0] += left_x * self.acceleration
            self.velocity[1] += left_y * self.acceleration
            
            # Apply friction
            self.velocity[0] *= self.friction
            self.velocity[1] *= self.friction
            
            # Update position
            self.dot_pos[0] += self.velocity[0] * self.dot_speed
            self.dot_pos[1] += self.velocity[1] * self.dot_speed
            
            # Keep dot within bounds
            self.dot_pos[0] = max(self.dot_radius, min(WIDTH - self.dot_radius, self.dot_pos[0]))
            self.dot_pos[1] = max(self.dot_radius, min(HEIGHT - self.dot_radius, self.dot_pos[1]))
            
            # Change dot size with triggers (L2/R2)
            l2 = (self.controller.get_axis(4) + 1) / 2  # Convert from -1,1 to 0,1
            r2 = (self.controller.get_axis(5) + 1) / 2
            
            if l2 > 0.1:  # Decrease size
                self.dot_radius = max(5, self.dot_radius - l2)
            if r2 > 0.1:  # Increase size
                self.dot_radius = min(50, self.dot_radius + r2)
    
    def draw_environment(self):
        # Draw background
        screen.fill(WHITE)
        
        # Draw boundary
        pygame.draw.rect(screen, BLACK, (0, 0, WIDTH, HEIGHT), 2)
        
        # Draw dot
        pygame.draw.circle(screen, self.dot_color, 
                         (int(self.dot_pos[0]), int(self.dot_pos[1])), 
                         int(self.dot_radius))
        
        # Draw instructions
        font = pygame.font.Font(None, 24)
        instructions = [
            "Left Stick: Move dot",
            "L2: Decrease dot size",
            "R2: Increase dot size",
            "△: Change color",
            "×: Reset position"
        ]
        
        for i, text in enumerate(instructions):
            surface = font.render(text, True, BLACK)
            screen.blit(surface, (10, 10 + i * 25))
    
    def handle_button_events(self, event):
        if event.type == pygame.JOYBUTTONDOWN:
            # Triangle button (3) - Change color
            if event.button == 3:
                self.dot_color = (
                    random.randint(0, 255),
                    random.randint(0, 255),
                    random.randint(0, 255)
                )
            # Cross button (0) - Reset position
            elif event.button == 0:
                self.dot_pos = [WIDTH // 2, HEIGHT // 2]
                self.velocity = [0, 0]
    
    def run(self):
        running = True
        clock = pygame.time.Clock()
        
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                self.handle_button_events(event)
            
            self.update_dot_position()
            self.draw_environment()
            pygame.display.flip()
            clock.tick(60)
        
        pygame.quit()

if __name__ == "__main__":
    import random  # For random colors
    environment = DotEnvironment()
    environment.run()

Connected to: DualSense Wireless Controller
